# Analyse how well we're doing guessing words
What tactics are working, am I improving, when is it difficult, etc.

In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [ ]:
load_dotenv()
PLAYERNAME = os.getenv('playername')

In [ ]:
database_url = os.getenv('PROD_DATABASE_URL').replace('postgresql', 'postgresql+psycopg')
engine = create_engine(database_url)

In [ ]:
database_url_dev = os.getenv('DATABASE_URL').replace('postgresql', 'postgresql+psycopg')
engine_dev = create_engine(database_url_dev)

In [ ]:
with engine.connect() as conn:
    games = pd.read_sql_query('SELECT * FROM woordrader.games', con=conn, index_col='game_id')
    positions = pd.read_sql_query('SELECT * FROM woordrader.shownletters', con=conn, index_col='letterplacement_id')
    boughtletters = pd.read_sql_query('SELECT * FROM woordrader.boughtletters', con=conn, index_col='buyevent_id')
    guesses = pd.read_sql_query('SELECT * FROM woordrader.guesses', con=conn, index_col='guess_id')

In [ ]:
position_relevant = (positions
                     .groupby('game_id')
                     .agg(LettersCorrect = pd.NamedAgg('correct', 'sum'),
                          LettersUnknown = pd.NamedAgg('shown_letter', lambda s: (s == '-').sum()),
                          )
                     )
buyevents_relevant = boughtletters.groupby('game_id').size().rename('LettersBought')
guesses_relevant = (guesses.set_index('game_id')
                    .rename(columns={'correct': 'GuessCorrect'})
                    [['guess_time', 'GuessCorrect']]
                    )                           
df = (games
      .join(position_relevant)
      .join(buyevents_relevant)
      .join(guesses_relevant, how='inner')
      .query('playername == @PLAYERNAME')
      )
df